# Example usage

To use `lisatools` in a project:

In [1]:
import lisatools

Standard library packages required to run this notebook:

In [2]:
import copy
import datetime

The `Fund` class stores general details about funds, such as their unique
ISIN identifier, the price, and the date the price was last updated.

In [3]:
ftse_global = lisatools.Fund(
    "FTSE Global All Cap Index Fund",
    172.14,
    isin="GB00BD3RZ582",
    date=datetime.date(2022, 11, 21)
)
print(ftse_global)

Fund('FTSE Global All Cap Index Fund', 172.14, date=datetime.date(2022, 11, 21), isin='GB00BD3RZ582')


The `Portfolio` class represents the funds actually being held. Each line item
in the portfolio is represented as a `Holding`, and stores the fund details,
the number of units held, and the target allocation fraction.

In [13]:
gilts = lisatools.Fund(
    "U.K. Gilt UCITS ETF (VGOV)",
    18.58,
    isin="IE00B42WWV65",
    date=datetime.date(2022, 11, 21)
)
holding1 = lisatools.Holding(ftse_global, 1.0, 0.6)
holding2 = lisatools.Holding(gilts, 5.0, 0.4)
pf = lisatools.Portfolio([holding1, holding2])
print(pf)

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   1.0000   172.14 0.6000 GB00BD3RZ582 2022-11-21
U.K. Gilt UCITS ETF (VGOV)       5.0000    92.90 0.4000 IE00B42WWV65 2022-11-21


The total value of a holding is equal to the number of units held times the unit price.

In [14]:
f = lisatools.Fund(price = 2.0)
h = lisatools.Holding(f, units = 3.0)

h.value()

6.0

The example portfolio above requires some trades to be made in order to reach
the target portfolio as defined by the `target_fraction` attributes of its
`Holding`s.

This target portfolio can be extracted with the `target_portfolio()` method.

In [15]:
print(pf.target_portfolio())

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.9238   159.02 0.6000 GB00BD3RZ582 2022-11-21
U.K. Gilt UCITS ETF (VGOV)       5.7059   106.02 0.4000 IE00B42WWV65 2022-11-21


The trades required to reach the target portfolio can be calculated using the
`trade_to_target()` method.

In [16]:
buy, sell = pf.trade_to_target()
print("Buy:\n=====", buy, "\nSell:\n=====", sell, sep = "\n")

Buy:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
U.K. Gilt UCITS ETF (VGOV)       0.7059    13.12 0.4000 IE00B42WWV65 2022-11-21

Sell:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.0762    13.12 0.6000 GB00BD3RZ582 2022-11-21


We can generate buying instructions when we want to add new money to the 
portfolio by first making a dummy cash holding, and pretending to "sell" it.

To avoid making changes to the original portfolio, we must make a deep copy.

In [17]:
cash = lisatools.Fund("Cash")
cash_holding = lisatools.Holding(cash, units = 100)
new_pf = copy.deepcopy(pf)
new_pf.append(cash_holding)
print(new_pf)

Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   1.0000   172.14 0.6000 GB00BD3RZ582 2022-11-21
U.K. Gilt UCITS ETF (VGOV)       5.0000    92.90 0.4000 IE00B42WWV65 2022-11-21
Cash                           100.0000   100.00 0.0000 None         2022-11-22


Note that the dummy cash holding has its last-updated date set to today.

In [18]:
buy, sell = new_pf.trade_to_target()
print("Buy:\n=====", buy, "\nSell:\n=====", sell, sep = "\n")

Buy:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
FTSE Global All Cap Index Fund   0.2724    46.88 0.6000 GB00BD3RZ582 2022-11-21
U.K. Gilt UCITS ETF (VGOV)       2.8588    53.12 0.4000 IE00B42WWV65 2022-11-21

Sell:
=====
Description                       Units    Value Target ISIN         Date
------------------------------ -------- -------- ------ ------------ ----------
Cash                           100.0000   100.00 0.0000 None         2022-11-22
